In [ ]:
%matplotlib inline

# Make container wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## Learned all of this from https://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

# Display all text in a text without truncation
pd.set_option('display.max_colwidth', -1)

Let's get the database credentials we need.

In [ ]:
import os

OSM_USER = os.getenv('POSTGRES_USER')
OSM_PASS = os.getenv('POSTGRES_PASS')

# This is a hack to be able to run osm2pgsql without interactively entering the password
os.environ['PGPASSWORD'] = OSM_PASS

--hstore
adds any tags not already in a conventional column to a hstore column. With the standard stylesheet this would result in tags like highway appearing in a conventional column while tags not in the style like name:en or lanes:forward would appear only in the hstore column.

--hstore-add-index
adds an index to the hstore columns

In [ ]:
%%bash 

osm2pgsql --hstore --latlong \
    --database osm \
    --username $POSTGRES_USER \
    --host osm_db \
    /data/saint_kitts_and_nevis-latest.osm.pbf

In [ ]:
import geopandas as gpd
import sqlalchemy
import qgrid

In [ ]:
grid_options = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    # Set to false if you want to scroll horizontally (so content fits better)
    'forceFitColumns': True,
    'defaultColumnWidth': 90,
    'rowHeight': 30,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,

    'sortable': True,
    'filterable': False,
    'highlightSelectedCell': True,
    'highlightSelectedRow': True
}
qgrid.set_defaults(grid_options=grid_options)

In [ ]:
connection_string = 'postgresql://{}:{}@osm_db:5432/osm'.format(OSM_USER, OSM_PASS)

In [ ]:
engine = sqlalchemy.create_engine(connection_string)

# What are the OSM Tables?
- planet_osm_point: which contains points of interest such as restaurants, hospitals, schools, supermarkets and addresses
- planet_osm_lines: contains roads and streets
- planet_osm_polygons: contains lakes, building footprints, administrative boundaries such as towns and cities

**Source: https://www.bostongis.com/PrinterFriendly.aspx?content_name=loading_osm_postgis**

# How do I extract values from the tags column?
In this example, we are extracting the value of the key `website` from the tags column.
```sql
SELECT osm_id, tags ->'website' AS website 
FROM planet_osm_polygon 
WHERE tags -> 'website' is not null limit 20;
```

# How do I check that there are values in the tags column?

```sql
SELECT * 
FROM planet_osm_polygon
WHERE array_length(akeys(tags), 1) IS NOT NULL
```

# How do I check all distinct tag keys?

```sql
SELECT DISTINCT skeys(tags) AS k
FROM planet_osm_polygon
ORDER BY skeys(tags) ASC
```

# Cool stuff with opening times
https://openingh.ypid.de/ is an interesting link that visualizes the opening hours tags. 

So far I've gathered
- `service_times` - it's recommended to use opening_hours. Used if times of service given is different from the opening hours. See [here](https://wiki.openstreetmap.org/wiki/Key:service_times) for more information
- `opening_hours`

In [ ]:
query = """
SELECT osm_id,
       tags -> 'addr:city'       AS "addr:city",
       tags -> 'addr:full'     AS "addr:full",
       "addr:housename",
       "addr:housenumber",
       tags -> 'addr:interpolation'   AS "addr:interpolation",
       tags -> 'addr:place' AS "addr:place",
       tags -> 'addr:postcode'   AS "addr:postcode",
       tags -> 'addr:street'     AS "addr:street",
       amenity,
       building,
       tags -> 'building:levels' AS building_levels,
       -- See https://wiki.openstreetmap.org/wiki/Key:level
       tags -> 'level' AS level,
       tags -> 'internet_access' AS internet_access,
       leisure,
       name,
       office,
       tags -> 'opening_hours' AS opening_hours,
       tags -> 'service_times' AS service_times,
       religion,
       service,
       shop,
       sport,
       surface,
       tourism,
       tags -> 'phone'           AS phone_number,
       tags -> 'website'         AS website,
       --tags -> 'website_1'         AS website_1,
       tags -> 'roof'            AS roof
       --,tags
       ,way
FROM planet_osm_polygon
WHERE
   tags -> 'addr:full' IS NOT NULL
   OR "addr:housename" IS NOT NULL
   OR "addr:housenumber" IS NOT NULL
   OR tags -> 'addr:place' IS NOT NULL
   OR tags -> 'addr:postcode' IS NOT NULL
   OR tags -> 'addr:street' IS NOT NULL
   OR amenity IS NOT NULL
   OR building IS NOT NULL
   OR tags -> 'building:levels' IS NOT NULL
   OR tags -> 'level' IS NOT NULL
   OR tags -> 'internet_access' IS NOT NULL
   OR leisure IS NOT NULL
   OR name IS NOT NULL
   OR office IS NOT NULL
   OR tags -> 'opening_hours' IS NOT NULL
   OR religion IS NOT NULL
   OR service IS NOT NULL
   OR tags -> 'service_times' IS NOT NULL
   OR shop IS NOT NULL
   OR sport IS NOT NULL
   OR tourism IS NOT NULL
   OR tags -> 'phone' IS NOT NULL
   OR tags -> 'website' IS NOT NULL
   OR tags -> 'website_1' IS NOT NULL
ORDER BY RANDOM()
"""
osm_polygon = gpd.read_postgis(sql=query,
                con=engine, geom_col='way', index_col='osm_id')

In [ ]:
# fillna method makes it so "None" is not displayed on the table
qgrid.show_grid((osm_polygon
                     .drop(columns=['way'])
                     .fillna('')
                ))

In [ ]:
osm_polygon.envelope.plot(edgecolor='red', facecolor='none')

In [ ]:
query = """
SELECT osm_id,
       tags -> 'addr:city'       AS "addr:city",
       tags -> 'addr:full'     AS "addr:full",
       "addr:housename",
       "addr:housenumber",
       tags -> 'addr:interpolation'   AS "addr:interpolation",
       tags -> 'addr:place' AS "addr:place",
       tags -> 'addr:postcode'   AS "addr:postcode",
       tags -> 'addr:street'     AS "addr:street",
       amenity,
       building,
       tags -> 'building:levels' AS building_levels,
       -- See https://wiki.openstreetmap.org/wiki/Key:level
       tags -> 'level' AS level,
       tags -> 'internet_access' AS internet_access,
       leisure,
       name,
       office,
       tags -> 'opening_hours' AS opening_hours,
       tags -> 'service_times' AS service_times,
       religion,
       service,
       shop,
       sport,
       surface,
       tourism,
       tags -> 'phone'           AS phone_number,
       tags -> 'website'         AS website,
       --tags -> 'website_1'         AS website_1,
       tags -> 'roof'            AS roof
       --,tags
       ,way
FROM planet_osm_point
WHERE
   tags -> 'addr:full' IS NOT NULL
   OR "addr:housename" IS NOT NULL
   OR "addr:housenumber" IS NOT NULL
   OR tags -> 'addr:place' IS NOT NULL
   OR tags -> 'addr:postcode' IS NOT NULL
   OR tags -> 'addr:street' IS NOT NULL
   OR amenity IS NOT NULL
   OR building IS NOT NULL
   OR tags -> 'building:levels' IS NOT NULL
   OR tags -> 'level' IS NOT NULL
   OR tags -> 'internet_access' IS NOT NULL
   OR leisure IS NOT NULL
   OR name IS NOT NULL
   OR office IS NOT NULL
   OR tags -> 'opening_hours' IS NOT NULL
   OR religion IS NOT NULL
   OR service IS NOT NULL
   OR tags -> 'service_times' IS NOT NULL
   OR shop IS NOT NULL
   OR sport IS NOT NULL
   OR tourism IS NOT NULL
   OR tags -> 'phone' IS NOT NULL
   OR tags -> 'website' IS NOT NULL
   OR tags -> 'website_1' IS NOT NULL
ORDER BY RANDOM()
"""
osm_point = gpd.read_postgis(sql=query,
                con=engine, geom_col='way',
                index_col='osm_id')

In [ ]:
qgrid.show_grid((osm_point
                     .drop(columns=['way'])
                     .fillna('')
                ))

In [ ]:
query = """
SELECT osm_id, "addr:housename", "addr:housenumber", amenity, building, 
    leisure, name, office, religion, 
    service, shop, sport, surface, tourism, 
    way, tags ->'website' AS website, tags
        FROM planet_osm_line
            WHERE name IS NOT NULL 
                OR tags ->'website' IS NOT NULL
                OR amenity IS NOT NULL
                OR building IS NOT NULL
                OR leisure IS NOT NULL
                OR office IS NOT NULL
                OR religion IS NOT NULL
                OR service IS NOT NULL
                OR shop IS NOT NULL
                OR sport IS NOT NULL
                OR tourism IS NOT NULL
                ORDER BY RANDOM()
"""
osm_line = gpd.read_postgis(sql=query,
                con=engine, geom_col='way',
                index_col='osm_id')

In [ ]:
qgrid.show_grid(osm_line.drop(columns=['way']))